# Alachua County restaurant inspection analysis
_This will take restaurant inspection data by the state of Florida and format it in a more reader-friendly way for publication in print and online. We'll filter for the most egregious current violations at restaurants in Alachua County._

After importing Pandas,etc., this reads in the state summary report year-to-date for District 5, which includes Alachua County, and adds an exception in case the file is not found (output probably needs to be set as variable so it can be written into the output file). The raw file has no headers and 82 columns. So this removes all but five columns and adds headers for those. Finally, it displays the first five rows of values.

In [1]:
import pandas as pd
import bs4
import datetime
import numpy as np
from urllib.request import urlopen
from urllib.error import HTTPError

In [2]:
try:
    insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                               usecols=[2,14,18,80,81])
except IOError:
    print("The file is not accessible.")
insp.columns = ["CountyName", "InspectDate", 
                "NumHighVio", "LicenseID", "VisitID"]

In [3]:
# filter for alachua county restaurants
alachua = insp[insp.CountyName == 'Alachua']

In [4]:
# filter for restaurants that had at least one serious violation
alachua = alachua[alachua.NumHighVio > 0]

In [5]:
# change date string to date object
alachua['InspectDate'] = pd.to_datetime(alachua['InspectDate'])
# sort most recent
alachua = alachua.sort_values('InspectDate', ascending=False)

__Goal with this next is to select date range prior to 'today', but this is hard coded now.__

In [6]:
today = pd.to_datetime('today')
startDay = datetime.date.today() - datetime.timedelta(days=30)
## want to get user input for timedelta 
alachua = alachua[(alachua['InspectDate'] > startDay) & (alachua['InspectDate'] < today)]

In [7]:
# takes LicenseID and VisitID, passes it into the urls for detailed reports
for index, rows in alachua.iterrows():
    visitID = rows['VisitID']
    licID = rows['LicenseID']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID= %s &licid= %s" % (visitID, licID)
    urls = urls.replace(' ', '')

__Now it's time to use those urls to access the detailed reports with inspector comments, and scrape those. This will need to be a "for loop" that cycles through all urls and then puts the output variables somehow, like a database?__

In [16]:
# need a for loop somewhere
# def get_inspect_detail(urls):   
html = urlopen(urls)
soup = bs4.BeautifulSoup(html.read(), 'html.parser')
details = soup.findAll('font', {'face':'verdana'})[10:]
    
siteName = details[0].text
licNum = details[2].text
siteRank = details[4].text
expDate = details[6].text
primeStatus = details[8].text
secStatus = details[10].text
siteAddress = details[12].text
inspectResult = details[20].text
observed = details[34].text

detailsLib = {
    'Restaurant': siteName,
    'License': licNum,
    'Rank': siteRank,
    'Expires': expDate,
    'Primary': primeStatus,
    'Secondary': secStatus,
    'Address': siteAddress,
    'Result': inspectResult,
    'Observed': observed
    }

print(detailsLib['Observed'].title())


																							
							Basic - Carbon Dioxide/Helium Tanks Not Adequately Secured.
 Observed Bulk Tank Outside Not Secured. A Braided Multi Stranded Observed Lack Of Back Flow Device On The Non Chemical Side Of The Splitter In The Mop Sink. Was Added. **Corrected On-Site**


In [41]:
html = urlopen('https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6509808&licid=5399007')
soup = bs4.BeautifulSoup(html.read(), 'lxml')
details = soup.findAll('font', {'face':'verdana'})[0:]
print(details)

#more of a regex job but less dependent on sit layout

[<font face="verdana" size="2"><b>    <img align="absmiddle" alt="" border="0" class="leftNav" src="/images/off.gif"/> Log On</b></font>, <font face="verdana" size="2"><b>Home</b></font>, <font face="verdana">Search for a Licensee</font>, <font face="verdana">Apply for a License</font>, <font face="verdana">View Application Status</font>, <font face="verdana">Find Exam Information</font>, <font face="verdana">File a Complaint</font>, <font face="verdana">AB&amp;T Delinquent<br/>    Invoice &amp; Activity<br/>    List Search</font>, <font face="verdana" size="-1"><strong>Each inspection report is a "snapshot" of conditions present at the time of the inspection.  By using this search, you are agreeing to our <a href="insptermsofuse.asp">Terms of Use</a>.</strong><p></p>
</font>, <font face="verdana" size="2">Name:</font>, <font face="verdana" size="2"><strong>HCA PUTNAM COMM HOSP 2</strong></font>, <font face="verdana" size="2">License Number:</font>, <font face="verdana" size="2"><stron